In [1]:
import os
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

Kode di atas mengimpor library Python yang digunakan untuk pemrosesan citra, analisis data, dan machine learning.

In [2]:
def insertGambar(folder):
    data = []
    filenames = []
    datalabel = [] 
    path = []
    data_distribution = {}
    for label in (os.listdir(folder)):
        images = os.listdir(folder+label)
        data_distribution[label] = len(images)
        datalabel.append(label)
        print(label)
        count = 0
        for filename in tqdm(os.listdir(folder+label)):
            if count == 100:
                break
            else :
                count += 1
            img = cv.imread(os.path.join(folder+label,filename))
            if img is not None:
                img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
                filenames.append(filename)
                path.append(label)
                data.append(img)
    return data, filenames, datalabel, path , data_distribution

Fungsi insertGambar membaca gambar dari folder yang berisi subfolder dengan label berbeda. Gambar-gambar tersebut dikonversi ke format RGB dan disimpan dalam daftar bersama dengan nama file dan labelnya. Fungsi ini juga membatasi jumlah gambar yang diambil per label hingga 100 dan menghitung distribusi data untuk setiap label. Hasilnya adalah kumpulan data gambar, nama file, label, jalur label, dan distribusi data.

In [3]:
data, filenames,label, path, distribusi = insertGambar("dataset/") 
print("Data: ", len(data))
print("Label: ", len(label))
print("Filenames: ", len(filenames))
print("Data Distribution: ", distribusi)

paper


100%|██████████| 100/100 [00:00<00:00, 373.60it/s]


plastic


100%|██████████| 100/100 [00:00<00:00, 439.35it/s]


trash


100%|██████████| 100/100 [00:00<00:00, 414.23it/s]

Data:  300
Label:  3
Filenames:  300
Data Distribution:  {'paper': 100, 'plastic': 100, 'trash': 100}


Fungsi insertGambar membaca gambar dari folder "dataset/", mengembalikan data gambar, nama file, label, jalur label, dan distribusi data. Kode ini kemudian mencetak jumlah gambar, jumlah label, jumlah nama file, dan distribusi data per label.

In [4]:
def grayScaling(img):
    gaussKernel = np.ones((3,3))/9
    r = img[:,:,2]
    g = img[:,:,1]
    b = img[:,:,0]
    filteredR = cv.filter2D(r,-1,gaussKernel)
    filteredG = cv.filter2D(g,-1,gaussKernel)
    filteredB = cv.filter2D(b,-1,gaussKernel)
    return np.round(filteredR/3 + filteredG/3 + filteredB/3).astype(np.uint8)

def grayScalingData(data):
    gray_data = []
    for i in tqdm(range(len(data))):
        gray_data.append(grayScaling(data[i]))
    return gray_data

Fungsi grayScaling mengonversi gambar berwarna menjadi grayscale dengan mengaplikasikan filter Gaussian pada setiap kanal warna dan menggabungkannya. Fungsi grayScalingData menerapkan grayScaling pada setiap gambar dalam data dan menyimpan hasilnya dalam gray_data.

In [5]:
grayData = grayScalingData(data)

100%|██████████| 300/300 [00:03<00:00, 78.22it/s] 


kode di atas adalah penerapan convert ke grayscale data yang diikuti dengan menampilkan sample data.

In [6]:
def matriksTransformasi(matriks):
    hasil = np.zeros(matriks.shape)
    for i in range(matriks.shape[0]):
        for j in range(matriks.shape[1]):
            hasil[i][j] = matriks[j][i]
    return hasil
def sum(matriks):
    hasil = 0
    for i in range(len(matriks)):
        for j in range(len(matriks[0])):
            hasil += matriks[i][j]
    return hasil

def derajat(img, derajat):
    max = np.max(img)
    temp=np.zeros([max+1,max+1])
    if derajat == 0:
        for i in range (len(img)):
            for j in range (len(img[0])-1):
                temp[img[i,j],img[i,j+1]] += 1
    elif derajat == 45:
        for i in range (len (img)-1):
            for j in range (len (img[0])-1):
                temp[img[i+1,j],img[i,j+1]] += 1
    elif derajat == 90:
        for i in range (len (img)-1):
            for j in range (len (img[0])):
                temp[img[i+1,j],img[i,j]] += 1
    elif derajat == 135:
        for i in range (len (img)-1):
            for j in range (len (img[0])-1):
                temp[img[i,j],img[i+1,j+1]] += 1
    hasil = temp+matriksTransformasi(temp)
    total = sum(hasil)
    for i in range (len (hasil)):
        for j in range (len (hasil)):
            hasil[i,j]/=total
    return hasil

def ekstraksi(image):
    hasil = []
    for img in tqdm(image):
        data = []
        data.append(derajat(img,0))
        data.append(derajat(img,45))
        data.append(derajat(img,90))
        data.append(derajat(img,135))
        hasil.append(data)    
    return hasil

Fungsi derajat menghitung matriks co-occurrence untuk gambar pada sudut tertentu dan menormalkannya. Fungsi ekstraksi menerapkan derajat pada setiap gambar dalam image untuk berbagai sudut dan menyimpan hasilnya dalam hasil.

In [7]:
ekstraksiData = ekstraksi(grayData)

100%|██████████| 300/300 [03:59<00:00,  1.25it/s]


kode di atas adalah penerapan fungsi ekstraksi pada data gray yang digunakan.

In [8]:
def contrast (data):
    hasil = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            hasil+=data[i,j]*pow(i-j,2)
    return hasil
def dissimilarity(data):  
    hasil = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            hasil+=data[i,j]*abs(i-j)
    return hasil
def homogeneity(data):
    hasil = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            hasil+=(data[i,j]/(1+pow(i-j,2)))
    return hasil
def energy(data):
    hasil = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            hasil += data[i][j] ** 2
    return hasil

def correlation(data):
    mean_x = 0
    mean_y = 0
    std_x = 0
    std_y = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            mean_x += i * data[i][j]
            mean_y += j * data[i][j]
    for i in range(len(data)):
        for j in range(len(data[0])):
            std_x += (i - mean_x) ** 2 * data[i][j]
            std_y += (j - mean_y) ** 2 * data[i][j]
    std_x = math.sqrt(std_x)
    std_y = math.sqrt(std_y)
    hasil = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            hasil += (i - mean_x) * (j - mean_y) * data[i][j]
    hasil /= std_x * std_y
    return hasil

def entropy(data):
    entropy = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            if data[i][j] > 0.0:
                entropy += -(data[i][j] * math.log(data[i][j]))
    return entropy
def asm(data):
    asm = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            asm += data[i][j] ** 2
    return asm

In [9]:
def ekstraksiFitur(data, path):
    hasil = []
    batas_gambar = len(data)
    for i in tqdm(range(batas_gambar)):
        extracted_data = [path[i]]
        for j in range(len(data[i])):
            contrast_val = contrast(data[i][j])
            extracted_data.append(contrast_val)
        for j in range(len(data[i])):
            dissimilarity_val = dissimilarity(data[i][j])
            extracted_data.append(dissimilarity_val)
        for j in range(len(data[i])):
            homogeneity_val = homogeneity(data[i][j])
            extracted_data.append(homogeneity_val)
        for j in range(len(data[i])):
            energy_val = energy(data[i][j])
            extracted_data.append(energy_val)
        for j in range(len(data[i])):
            correlation_val = correlation(data[i][j])
            extracted_data.append(correlation_val)
        for j in range(len(data[i])):
            asm_val = asm(data[i][j])
            extracted_data.append(asm_val)
            entropy_val = entropy(data[i][j])
            extracted_data.append(entropy_val)
        hasil.append(extracted_data)
    return hasil

In [10]:
angles = ['0', '45', '90', '135']
fiturs = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
headers = ['path']
for fitur in fiturs:
    headers.extend([f'{fitur}_{angle}' for angle in angles])
headers.extend([f'asm_{angle}' for angle in angles])
headers.extend([f'entropy_{angle}' for angle in angles])

In [11]:
hasil = ekstraksiFitur(ekstraksiData, path)
df = pd.DataFrame(hasil, columns=headers)
df

100%|██████████| 300/300 [05:12<00:00,  1.04s/it]


,path,contrast_0,contrast_45,contrast_90,contrast_135,dissimilarity_0,dissimilarity_45,dissimilarity_90,dissimilarity_135,homogeneity_0,...,correlation_90,correlation_135,asm_0,asm_45,asm_90,asm_135,entropy_0,entropy_45,entropy_90,entropy_135
0,paper,34.627981,65.114734,31.449973,62.041637,1.953303,2.805322,1.718842,2.611155,0.713239,...,0.993843,0.987868,0.010774,6.293078,0.009106,6.579898,0.010514,6.261537,0.009089,6.544980
1,paper,22.833313,24.654034,3.293407,26.701011,2.510284,2.726927,1.191559,2.855370,0.449226,...,0.999565,0.996460,0.004571,7.319258,0.004467,7.409926,0.005112,6.751692,0.004443,7.457003
2,paper,151.411983,197.678054,125.131573,318.460204,6.856129,7.162595,6.083913,9.886068,0.379972,...,0.986802,0.966409,0.001399,8.157475,0.001278,8.186649,0.001515,8.041390,0.001091,8.474081
3,paper,6.601659,12.791383,4.014503,6.934772,0.729829,0.989990,0.551607,0.771717,0.792864,...,0.996767,0.994398,0.008173,5.534304,0.006777,5.709364,0.008912,5.391666,0.006911,5.650684
4,paper,42.516736,83.167102,43.673583,83.989244,2.422181,3.912203,2.600568,3.887831,0.646082,...,0.983117,0.967559,0.004873,6.488847,0.003663,6.868850,0.005105,6.503245,0.003659,6.860561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,trash,2.930793,6.022017,2.954359,5.272097,0.401337,0.605044,0.451809,0.637454,0.867713,...,0.999444,0.999008,0.047057,4.855375,0.043049,5.045561,0.047221,4.901966,0.042851,5.091149
296,trash,4.912921,12.474302,7.477684,11.149970,1.209368,1.761176,1.297569,1.747365,0.626781,...,0.998718,0.998087,0.030734,6.017515,0.028438,6.278786,0.030704,6.036419,0.028378,6.281896
297,trash,3.547512,4.821310,2.653389,7.056465,0.670983,0.810907,0.625525,1.057405,0.792475,...,0.999242,0.997984,0.017830,5.318440,0.015449,5.468583,0.018440,5.287173,0.015239,5.595473
298,trash,24.267444,50.958613,26.394684,45.554102,1.568682,2.320025,1.660891,2.305396,0.753475,...,0.993482,0.988751,0.010586,6.073735,0.008886,6.340741,0.010591,6.144308,0.008789,6.388647


In [12]:
df.to_csv('sebelum_preprocessing_100.csv', index=False)

In [13]:
from sklearn.decomposition import PCA
df = pd.read_csv('sebelum_preprocessing_100.csv')

X = df.drop('path', axis=1)
y = df['path']

pca = PCA(n_components=25)
x = pca.fit_transform(X)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_pred = knn_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_pred)
print("K-Nearest Neighbors (KNN) Accuracy:", knn_accuracy)

svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_pred)
print("Support Vector Machine (SVM) Accuracy:", svm_accuracy)

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
print("Random Forest Accuracy:", rf_accuracy)

K-Nearest Neighbors (KNN) Accuracy: 0.7
Support Vector Machine (SVM) Accuracy: 0.8
Random Forest Accuracy: 0.8166666666666667


In [16]:
models = {'KNN': knn_pred, 'SVM': svm_pred, 'Random Forest': rf_pred}
results = {}

for model_name, y_pred in models.items():
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    results[model_name] = [accuracy, precision, recall, f1]

results_df = pd.DataFrame(results, index=['Accuracy', 'Precision', 'Recall', 'F1-Score']).T
print(results_df)

               Accuracy  Precision    Recall  F1-Score
KNN            0.700000   0.734083  0.700000  0.701088
SVM            0.800000   0.800000  0.800000  0.800000
Random Forest  0.816667   0.838360  0.816667  0.820445


Melatih model KNN, SVM, dan Random Forest dengan data pelatihan (xTrain, yTrain). Memprediksi label untuk data pengujian (xTest) dengan masing-masing model dan menghitung metrik evaluasi: akurasi, presisi, recall, dan F1-Score. Metrik ini kemudian disimpan dalam DataFrame hasil dan dicetak untuk membandingkan performa masing-masing model. Pada akhirnya, didapatkan akurasi seperi pada gambar.